# Construcción de una Red Neuronal desde cero
A través de este tutorial, se va a construir una Red Neuronal Artificial (RNA) explicando de forma teórica los diferentes elementos necesarios para la construcción y entrenamiento del modelo. Además, se pondrá en práctica la red para evaluar su correcto funcionamiento.

# Índice de contenidos
- [1 - Inicialización de capas](#1)
- [2 - Inicialización de parámetros](#2)
- [3 - Implementar "forward propagation"](#3)
- [4 - Computación del coste](#4)
- [5 - Implementación del backward propagation](#5)
- [6 - Actualización de parámetros (gradient descent)](#6)
- [7 - Entrenamiento del modelo](#7)
- [8 - Predicción](#8)

# 1.- Instalación de librerías
En primer lugar, se instalan e importan todas las librerías necesarias para la realización de la implementación.

## 1.1.- Instalación de las librerías
Se va a emplear 'numpy' y 'matplotlib':

In [ ]:
!pip install numpy
!pip install matplotlib

## 1.2.- Inicialización de librerías
Una vez instaladas, se inicializan las librerías:

In [ ]:
import numpy as np
import matplotlib as plt

# 2.- Inicialización de capas
Este método nos permite definir la estructura de la red neuronal que vamos a construir. 

In [ ]:
def init_layers(X, Y):
    """
    Variables de entrada:
    X -- conjunto de datos de entrada (tamaño de entrada, número de ejemplos)
    Y -- etiquetas (tamaño de salida, número de ejemplos)
    
    Variables de salida:
    input_units -- número de neuronas en la capa de entrada (equivalente al número de variables de entrada)
    hidden_units -- númeor de neuronas en la capa oculta
    output_units -- número de neuronas en la capa de salida
    """
    
    input_units = X.shape[0]
    # hidden_units = 2 
    output_units = Y.shape[0]

    return (input_units, hidden_units, output_units)

# 3.- Inicialización de parámetros
Este método nos permite inicializar los parámetros para una red neuronal de `l` capas.

# 4.- Implementación de 'forward propagation'
Para implementar la propagación hacia delante, se deben de realizar los siguientes cálculos:
* Cálculo de los parámetros pre-activación.
* Cálculo de la activación de las neuronas.


## 4.1.- Parámetros pre-activación
Este método se encarga de calcular los parámetros que se introducen en la función de activación de la neurona. La ecuación que se emplea es:
$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$
donde ${[l]}$ es el número de capa; $W^{[l]}$ son los pesos actuales de la capa actual; $A^{[l-1]}$ es la activación de la capa anterior; $b^{[l]}$ es el bias de la capa.

In [ ]:
def calculo_preactivacion(A, W, b):
    """
    Implementación del cálculo de parámetros pre-activación de la neurona.

    Variables de entrada:
    A -- activación de la capa anterior.
    W -- matriz de pesos de la capa.
    b -- vector de bias.

    Variables de salida:
    Z -- entrada de la función de activación.
    cache -- tupla de Python con la activación de la capa, pesos de la capa y bias. Será empleado más adelante.
    """
    Z = np.dot(W,A)+b
    cache = (A, W, b)
    
    return Z, cache

## 4.2.- Activación de la capa
Este método se encargará de calcular la activación de la neurona ante los parámetros pre-activación. Son múltiples las opciones que podemos emplear para realizar este paso. A continuación, explicamos y definimos las más comunes:
* **Sigmoid**: 
* **ReLU**:
* **Softmax**: 
* **Leaky ReLU**: 
* **Tanh**: 

### 4.2.1.- Sigmoid
La función sigmoidal sigue la siguiente fórmula matemática:


In [ ]:
def sigmoid(z):
    activation = 1 / (1 +  np.exp(-z))

    return activation

### 4.2.2.- ReLU
La función ReLU sigue la siguiente fórmula matemática:

In [ ]:
def relu(z):
    activation = max(0,z)

    return activation

### 4.2.3.- Softmax
La función Softmax sigue la siguiente fórmula matemática:

In [ ]:
def relu(z):
    max_z = np.max(z)
    exp = np.exp(z-max_z)
    activation = exp/max_z

    return activation

### 4.2.4.- Leaky ReLU
La función Leaky ReLU sigue la siguiente fórmula matemática:

In [ ]:
def leaky_relu(z):
    activation = max(0.01*z,z)

    return activation

### 4.2.5.- Tanh
La función Tanh sigue la siguiente fórmula matemática:

In [ ]:
def tanh(z):
    activation = np.tanh(z)

    return activation

# 5.- Computación de la pérdida
En este punto, ya tenemos computada la activación de cada capa por lo que tan sólo quedaría computar el coste. El coste define el error existente entre las predicciones del modelo y las etiquetas reales del conjunto de datos. Para asegurarnos de que nuestro modelo está aprendiendo correctamente, el coste debería de reducirse en cada etapa del entrenamiento.

Procedemos por tanto a definir algunas de las funciones de pérdida (que son las funciones que calculan el 'coste' sobre un conjunto de datos) más comunes:
* **Cross-entropy**: 
* **Log Loss**: